<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [1]:
!pip install tweepy

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import tweepy

In [3]:
keys = pd.read_csv("config.csv", header=None) # Nice to do: Manage Environment variables
keys = dict(zip(keys[0],keys[1]))

In [4]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [19]:
tweets = tweepy.Cursor(api.search,
                       q=['venezuela -RT'],
                       lang='es',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [20]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append(tweet.user.location)
    temp.append(tweet.place)
    temp.append(tweet.coordinates)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [21]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'user_location', 'place', 'coordinates', 'id', 'link']
data

,tweet,date,user_location,place,coordinates,id,link
0,"El bolívar de Venezuela se devalúa un 20,24 % ...",2020-11-14 02:46:52,Venezuela,None,None,1327442850380996609,https://twitter.com/i/web/status/1327442850380...
1,@bdelamaza Todos los partidos son distintos. V...,2020-11-14 02:46:51,Huépil - Chile,Place(_api=<tweepy.api.API object at 0x1190e0f...,None,1327442843695255558,https://twitter.com/i/web/status/1327442843695...
2,"Pero no lo puedes sacar, porque te resuelve co...",2020-11-14 02:46:48,En el lado oscuro de la Luna.,None,None,1327442832261582849,https://twitter.com/i/web/status/1327442832261...
3,@Mayafernandeza Jajjajajjajaja las encuestas e...,2020-11-14 02:46:48,"Metropolitana de Santiago, Chile",None,None,1327442830709690368,https://twitter.com/i/web/status/1327442830709...
4,@vitorkkkk77 @FoxSportsBrasil Contra a forte V...,2020-11-14 02:46:47,,None,None,1327442829480763392,https://twitter.com/i/web/status/1327442829480...
...,...,...,...,...,...,...,...
95,"[#Qatar2022🏆🇶🇦] Con gol de Roberto Firmino, #B...",2020-11-14 02:45:30,"La Plata, Buenos Aires 🇦🇷",None,None,1327442504262832130,https://twitter.com/i/web/status/1327442504262...
96,@2010MisterChip Pero algunos creían que Venezu...,2020-11-14 02:45:30,"Maracaibo, Venezuela",None,None,1327442503331688448,https://twitter.com/i/web/status/1327442503331...
97,Sinceramente no me quejo del partido de Venezu...,2020-11-14 02:45:29,,None,None,1327442501490372609,https://twitter.com/i/web/status/1327442501490...
98,@CHVNoticias Debería haberse enfermado en Vene...,2020-11-14 02:45:25,,None,None,1327442485203890182,https://twitter.com/i/web/status/1327442485203...


## Pre-procesamiento para Twitter

In [22]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [23]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,user_location,place,coordinates,id,link,clean,links,hashtags,mentions
0,"El bolívar de Venezuela se devalúa un 20,24 % ...",2020-11-14 02:46:52,Venezuela,None,None,1327442850380996609,https://twitter.com/i/web/status/1327442850380...,"El bolívar de Venezuela se devalúa un 20,24 % ...",[https://t.co/227DL06996],[],[]
1,@bdelamaza Todos los partidos son distintos. V...,2020-11-14 02:46:51,Huépil - Chile,Place(_api=<tweepy.api.API object at 0x1190e0f...,None,1327442843695255558,https://twitter.com/i/web/status/1327442843695...,Todos los partidos son distintos. Venezuela t...,[],[],[@bdelamaza]
2,"Pero no lo puedes sacar, porque te resuelve co...",2020-11-14 02:46:48,En el lado oscuro de la Luna.,None,None,1327442832261582849,https://twitter.com/i/web/status/1327442832261...,"Pero no lo puedes sacar, porque te resuelve co...",[https://t.co/5rIm6vhqRV],[],[]
3,@Mayafernandeza Jajjajajjajaja las encuestas e...,2020-11-14 02:46:48,"Metropolitana de Santiago, Chile",None,None,1327442830709690368,https://twitter.com/i/web/status/1327442830709...,Jajjajajjajaja las encuestas en cuba y Venezuela,[],[],[@Mayafernandeza]
4,@vitorkkkk77 @FoxSportsBrasil Contra a forte V...,2020-11-14 02:46:47,,None,None,1327442829480763392,https://twitter.com/i/web/status/1327442829480...,Contra a forte Venezuela,[],[],"[@vitorkkkk77, @FoxSportsBrasil]"


In [24]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [25]:
data.head()

,tweet,date,user_location,place,coordinates,id,link,clean,links,hashtags,mentions,emojis
0,"El bolívar de Venezuela se devalúa un 20,24 % ...",2020-11-14 02:46:52,Venezuela,None,None,1327442850380996609,https://twitter.com/i/web/status/1327442850380...,"El bolívar de Venezuela se devalúa un 20,24 % ...",[https://t.co/227DL06996],[],[],[]
1,@bdelamaza Todos los partidos son distintos. V...,2020-11-14 02:46:51,Huépil - Chile,Place(_api=<tweepy.api.API object at 0x1190e0f...,None,1327442843695255558,https://twitter.com/i/web/status/1327442843695...,Todos los partidos son distintos. Venezuela t...,[],[],[@bdelamaza],[]
2,"Pero no lo puedes sacar, porque te resuelve co...",2020-11-14 02:46:48,En el lado oscuro de la Luna.,None,None,1327442832261582849,https://twitter.com/i/web/status/1327442832261...,"Pero no lo puedes sacar, porque te resuelve co...",[https://t.co/5rIm6vhqRV],[],[],[]
3,@Mayafernandeza Jajjajajjajaja las encuestas e...,2020-11-14 02:46:48,"Metropolitana de Santiago, Chile",None,None,1327442830709690368,https://twitter.com/i/web/status/1327442830709...,Jajjajajjajaja las encuestas en cuba y Venezuela,[],[],[@Mayafernandeza],[]
4,@vitorkkkk77 @FoxSportsBrasil Contra a forte V...,2020-11-14 02:46:47,,None,None,1327442829480763392,https://twitter.com/i/web/status/1327442829480...,Contra a forte Venezuela,[],[],"[@vitorkkkk77, @FoxSportsBrasil]",[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

## Cuales y cuantos emojis hay en los tweets

In [28]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
0,🇻🇪,12
7,🇧🇷,9
2,🆚,6
12,🇺🇾,4
24,🔥,4


## Veamos los emojis mas usados

In [31]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

Llama la atención que los tweets o las conversaciones de los tweets estan bastante relacionados con otros paises. Será necesario analizarlos para ver que de que se tratan.